In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import csv
import math
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import pytorch_lightning as pl
import glob
import torchvision.transforms.functional as TF
import random

## First, we slice the train images into 15 x 15 pixels with the ground truth in the middle

In [ ]:
# Used to fetch and save files in load_data()
def ndigit(n, x):
    x = str(x)
    while(len(x) < n):
        x = "0" + x
    return x

In [ ]:
# Function to load images, enrich them with moisture and vegetation index (i.e. increase channels from 10 to 12),
# extract ground truths from the masks, pad the images to work with ground truths close to the edges,
# slice images into 15 x 15 patches and save them.
def load_data(res, files = 20):
    j = 0
    path = ["02", "train"]
    res = int((res-1)/2)
    nan_values = 0

    # Load images and masks
    for p in path:
        for f in range(files):
            image = np.load(f"images_{p}/images/image_{ndigit(3, f)}.npy")
            mask = np.load(f"masks_{p}/masks/mask_{ndigit(3, f)}.npy")
            
            # In anticipation of toTensor() in transforms later which expects an array of H x W x C and converts it into C x H x W.
            image = np.transpose(image, (1,2,0))
            mask = np.transpose(mask, (1,2,0))
            
            nan_values_before = (np.count_nonzero(np.isnan(image)))
            
            # Extract spectral bands for calculating vegetation index
            channel8 = image[:, :, 6]
            channel4 = image[:, :, 2]

            # Calculate the vegetation index with small epsilon in order to prevent dividing through zero (which results in NaN values)
            vegetation_array = np.divide((np.subtract(channel8, channel4)), np.add(np.add(channel8, channel4), 1e-6))
            
            nan_values_vegetation = (np.count_nonzero(np.isnan(vegetation_array)))
            
            if(nan_values_vegetation > 0): 
                print("picture",f"images_{p}/images/image_{ndigit(3, f)}.npy had", nan_values_before, "before vegetation index")
                print("picture",f"images_{p}/images/image_{ndigit(3, f)}.npy has", nan_values_vegetation, "nan_values after adding vegetation")
            
            
            vegetation_array = np.nan_to_num(vegetation_array, nan=0.0)

            # Add vegetation index to the image as eleventh channel
            image_veg = np.concatenate((image, vegetation_array[:, :, np.newaxis]), axis=2)

            nan_values_before = 0
            
            # Extract spectral bands for calculating moisture index
            channel8a = image[:, :, 7]
            channel11 = image[:, :, 8]

            # Calculate the moisture index with small epsilon in order to prevent dividing through zero
            moisture_array = np.divide((np.subtract(channel8a, channel11)), np.add(np.add(channel8a, channel11), 1e-6))
            
            nan_values_moisture = (np.count_nonzero(np.isnan(moisture_array)))
            
            if(nan_values_moisture > 0): 
                print("picture",f"images_{p}/images/image_{ndigit(3, f)}.npy had", nan_values_before, "before moisture index")
                print("picture",f"images_{p}/images/image_{ndigit(3, f)}.npy has", nan_values_moisture, "nan_values after adding moisture")
            
            # Add moisture index to the image as twelfth channel
            image_veg_mois = np.concatenate((image_veg,moisture_array[:,:, np.newaxis]), axis = 2)

            nan_values_pic = np.count_nonzero(np.isnan(image_veg_mois))
            nan_values += nan_values_pic

            # Add padding to every image and mask edge in case there are ground truths which are too close to an edge
            padded_image = np.pad(image_veg_mois, ((res+1, res+1), (res+1, res+1), (0,0)), mode='constant')
            padded_mask = np.pad(mask, ((res+1, res+1), (res+1, res+1), (0,0)), mode='constant')

            # Extract ground truths
            ground_truths_pos = np.array(np.where(padded_mask != 0)).T
            
            # Slice and save patches around each ground truth
            for i in ground_truths_pos: 
                patch = (padded_image[i[0]-res : i[0]+res+1, i[1]-res : i[1]+res+1, :], padded_mask[i[0], i[1], 0])
                np.save(f"patches/train/patch_{p}_{ndigit(3, f)}_{ndigit(5, j)}.npy", np.array(patch, dtype="object"))                                 
                j += 1
    print("Added Vegetation (B8-B4)/(B8+B4)")
    print("Added Moisture (B8A-B11)/(B8A+B11)")
    print("Patched the pictures")
    print("NaN values:", nan_values)


In [ ]:
# Check number of ground truths
# num = 0
# path = ["02", "train"]
# for p in path:
#     for f in range(20):
#         mask = np.load(f"masks_{p}/masks/mask_{ndigit(3, f)}.npy")
#         ground_truths_pos = np.array(np.where(mask != 0)).T
#         num = num + len(ground_truths_pos)
# print(num)

In [2]:
res = 15
#load_data(res)

## Then, we load the data and have a look

In [3]:
# Load patches
directory = 'patches/train'
file_paths = glob.glob(directory + '/*.npy')
dataset = [np.load(file_path, allow_pickle=True) for file_path in file_paths]

In [4]:
# Check some metrics on the dataset
l_t = len(dataset)
X,y = dataset[0]
X_s = X.shape
y_s = y.shape
print(f"Trainset contains {l_t} samples where each input shape is {X_s} and target shape is {y_s}.")

Trainset contains 38863 samples where each input shape is (15, 15, 12) and target shape is ().


In [5]:
dataset[0]

array([array([[[7.10000000e+02, 9.44000000e+02, 1.12600000e+03, ...,
                1.73300000e+03, 3.53802009e-01, 5.10183505e-02],
               [7.73000000e+02, 9.66000000e+02, 1.10600000e+03, ...,
                1.73300000e+03, 3.61616162e-01, 5.10183505e-02],
               [5.86000000e+02, 8.70000000e+02, 9.98000000e+02, ...,
                1.69800000e+03, 4.78578892e-01, 1.13785970e-01],
               ...,
               [5.26000000e+02, 6.14000000e+02, 6.02000000e+02, ...,
                1.30600000e+03, 4.68901632e-01, 4.63042853e-02],
               [4.82000000e+02, 5.94000000e+02, 6.07000000e+02, ...,
                1.30600000e+03, 4.60444444e-01, 4.63042853e-02],
               [4.96000000e+02, 6.18000000e+02, 6.02000000e+02, ...,
                1.19300000e+03, 4.76976542e-01, 5.99423631e-02]],

              [[7.04000000e+02, 9.52000000e+02, 1.16000000e+03, ...,
                1.73300000e+03, 3.66639367e-01, 5.10183505e-02],
               [7.58000000e+02, 9.600000

In [6]:
# Calculate the sizes of the training set and validation set
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Split trainset into trainset and valset
trainset, valset = random_split(dataset, [train_size, val_size])
print(len(trainset), len(valset))

31090 7773


In [7]:
# stacks data for all train images in one array
inputs = np.stack([data[0] for data in trainset], axis=0)
inputs.shape

(31090, 15, 15, 12)

In [8]:
# Compute the mean and standard deviation for each channel from all pictures along heigth and width
channel_means = np.mean(inputs, axis=(0, 1, 2), keepdims=False)
channel_stds = np.std(inputs, axis=(0, 1, 2), keepdims=False)
channel_meds = np.median(inputs, axis=(0, 1, 2), keepdims=False)
channel_mins = np.min(inputs, axis=(0, 1, 2), keepdims=False)
channel_maxs = np.max(inputs, axis=(0, 1, 2), keepdims=False)
channel_10_quants = np.quantile(inputs, 0.1, axis=(0, 1, 2), keepdims=False)
channel_90_quants = np.quantile(inputs, 0.9, axis=(0, 1, 2), keepdims=False)

print(len(channel_means), len(channel_stds))
print(channel_means, channel_stds)

12 12
[4.99621724e+02 7.37692006e+02 7.11432591e+02 1.18871994e+03
 2.47144648e+03 2.93903173e+03 3.07567553e+03 3.20778523e+03
 2.04999702e+03 1.24576666e+03 6.29505878e-01 2.24810481e-01] [2.81440099e+02 3.47990360e+02 5.03348864e+02 5.03846522e+02
 6.36426717e+02 7.78754342e+02 8.43533804e+02 8.32242617e+02
 6.79728210e+02 5.87340485e+02 2.13783702e-01 1.43432713e-01]


In [10]:
for i in range(inputs.shape[3]):
  print(f"Channel {i+1}")
  print(f"mean = {channel_means[i]}")
  print(f"std = {channel_stds[i]}")
  print(f"median = {channel_meds[i]}")
  print(f"min = {channel_mins[i]}")
  print(f"max = {channel_maxs[i]}")
  print(f"10 percent quantile = {channel_10_quants[i]}")
  print(f"90 percent quantile = {channel_90_quants[i]}")
  print()

Channel 1
mean =  499.62172431292663
std =  281.44009926313737
median =  430.0
min =  0.0
max =  10912.0
10 percent quantile =  236.0
90 percent quantile =  866.0

Channel 2
mean =  737.6920064329366
std =  347.9903600970017
median =  694.0
min =  0.0
max =  11752.0
10 percent quantile =  381.0
90 percent quantile =  1212.0

Channel 3
mean =  711.4325905435832
std =  503.34886399265343
median =  550.0
min =  0.0
max =  13104.0
10 percent quantile =  228.0
90 percent quantile =  1464.0

Channel 4
mean =  1188.719939101533
std =  503.84652156095876
median =  1157.0
min =  0.0
max =  10560.0
10 percent quantile =  639.0
90 percent quantile =  1894.0

Channel 5
mean =  2471.44647996855
std =  636.4267173367139
median =  2543.0
min =  0.0
max =  9914.0
10 percent quantile =  1681.0
90 percent quantile =  3177.0

Channel 6
mean =  2939.0317329616523
std =  778.7543423428289
median =  2988.0
min =  0.0
max =  10352.0
10 percent quantile =  1977.0
90 percent quantile =  3857.0

Channel 7
mean 

In [11]:
# Create custom dataset class in order to transform dataset and apply data augmentation
class CustomDataset(Dataset):
    def __init__(self, dataset, transform, augmentations):
        self.dataset = dataset
        self.transform = transform
        self.augment = augmentations

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        data, target = self.dataset[index]

        # Apply transformations
        if self.transform:
            data = self.transform(data)

        # Apply augmentations
        if self.augment:
           data = self.augment(data)

        return data, target

In [25]:
# Custom rotation transformation from the documentation in order to rotate at given angles,
# not select from range of angles.
class MyRotationTransform:
    """Rotate by one of the given angles."""

    def __init__(self, angles):
        self.angles = angles

    def __call__(self, image):
        angle = random.choice(self.angles)
        return TF.rotate(image, angle)

# Custom elastic transformation which adds randomness. Originally, transforms.ElasticTransform transforms
# every image, but now only at given probability.
class RandomElasticTransform:
    def __init__(self, probability, alpha, sigma):
        self.probability = probability
        self.alpha = alpha
        self.sigma = sigma

    def __call__(self, image):
        if np.random.rand() < self.probability:
          elastic_transformer = transforms.ElasticTransform(self.alpha, self.sigma)
          return elastic_transformer(image)
        else:
          return image

# Custom normalization class. Only the first 10 channels need to be normalized, the last two are already in normalized form
# since their creation. The regular transforms.Normalize will throw an error, however, if less means/stds are passed than
# there are channels.
class MyNormalization:
    def __init__(self, means, stds):
        self.means = means
        self.stds = stds
        self.channels = [i for i in range(len(means))] # Channels to normalize

    def __call__(self, image):
        for i in self.channels:
            image[i, :, :] = (image[i, :, :] - self.means[i]) / self.stds[i]
        return image

In [26]:
transform = transforms.Compose(
    [transforms.ToTensor(), # if input is 3D array then toTensor() switches dimensions from H x W x C to C x H x W
     transforms.ConvertImageDtype(torch.float64),
     #transforms.Lambda(lambda x : x / 3000),
     #transforms.Lambda(lambda x : torch.where(x > 1, 1, x)), # fix pixel values between 0 and 1
     MyNormalization(means=channel_means[0:10], # applies normalization with means and stds of trainset
                          stds=channel_stds[0:10])
     ])

augmentations = transforms.Compose(
    [MyRotationTransform(angles=[0, 90, 180, 270, 0]),
     transforms.RandomAffine(degrees=0, translate=(0.2,0.2)), # shift in both directions along 0.5 * height on y-axis and 0.5 * width on x-axis
                                                                                 # scale in range 0.25 <= scale <= 0.75
     transforms.ElasticTransform(alpha=5.0, sigma=0.5), # displaces pixels
     transforms.RandomHorizontalFlip(), # default p = 0.5
     transforms.RandomVerticalFlip()
    ])

In [27]:
# Create the custom trainset
trainset_transformed = CustomDataset(trainset, transform=transform, augmentations=None)
valset_transformed = CustomDataset(valset, transform=transform, augmentations=None)
#trainset_transformed[0][0]
#on the fly augmentation during training, hence no additional pictures in trainset
#len(trainset_transformed)

In [28]:
trainset_transformed[0]

(tensor([[[-0.8230, -0.8727, -0.6062,  ..., -0.2083,  0.2039,  0.6231],
          [-0.7875, -0.9083, -0.7555,  ...,  0.2501,  0.4988,  0.6054],
          [-0.7164, -0.8656, -0.7875,  ...,  0.3424,  0.4348,  0.5592],
          ...,
          [ 1.0673,  0.9749,  0.8115,  ...,  0.4633,  0.4704,  0.3638],
          [ 1.0175,  0.9003,  0.8008,  ...,  0.3780,  0.4597,  0.3211],
          [ 1.0175,  0.9287,  0.8186,  ...,  0.3034,  0.4277,  0.4029]],
 
         [[-0.4963, -0.6256, -0.5250,  ...,  0.1791,  0.5957,  1.1934],
          [-0.4934, -0.7003, -0.6974,  ...,  0.9032,  1.2682,  1.3199],
          [-0.4158, -0.6572, -0.6486,  ...,  0.9865,  1.0584,  1.2480],
          ...,
          [ 1.6992,  1.5239,  1.5728,  ...,  1.3860,  1.3716,  1.0957],
          [ 1.5613,  1.5182,  1.6187,  ...,  1.1791,  1.1791,  1.0067],
          [ 1.3630,  1.6791,  1.6676,  ...,  1.0153,  1.1274,  1.0670]],
 
         [[-0.7339, -0.8075, -0.5015,  ..., -0.0108,  0.5435,  0.8256],
          [-0.6585, -0.8134,

In [ ]:
# Create data loaders for transformed training set and validation set
batch_size = 64
trainloader = DataLoader(trainset_transformed, batch_size=batch_size, shuffle=True, num_workers=2)
validloader = DataLoader(valset_transformed, batch_size=batch_size, shuffle=False, num_workers=2)


In [ ]:
#limit = 0
#for batch in trainloader:

    #while(limit <1):
        #print(batch)
    #    print(x.shape)
    #    print(x.size)
    #    print(y.shape)
    #    print(y)
        #limit += 1

## Next, we define the model and train it

In [ ]:
class MyCNNModel(pl.LightningModule): # New! def init(self, layers, lr=0.01, classes=None): super().init() # <- Very important! self.lr = lr self.classes = classes ## Build model self.layers = nn.Sequential(layers) # Create a sequential model

    def __init__(self, *layers, classes=None):
        super().__init__()

        self.lr = 0.01  # Assign the learning rate here
        self.classes = classes

        self.layers = nn.Sequential(*layers)  # Create a sequential model
        
    def forward(self, X):
        return self.layers(X)

    def predict(self, X):
        with torch.no_grad():
            y_hat = self(X).argmax(1)
        if self.classes is not None:
            y_hat = [self.classes[i] for i in y_hat]
        return y_hat

    def training_step(self, batch, batch_idx, log_prefix='train'):
        X, y = batch
        y_hat = self(X)
        y_hat = y_hat.flatten()
        loss = nn.MSELoss()
        loss = loss(y_hat, y)
        self.log(f"{log_prefix}_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        with torch.no_grad():
            return self.training_step(batch, batch_idx, log_prefix='valid')

    def configure_optimizers(self):
        # Adam with Weight Decay
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=0.01)

        # ReduceLROnPlateau reduces the learning rate by 0.1 if the val_loss has not decreased within the last 10 epochs.
        scheduler = {
            "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, verbose=True),
            # 'step' updates the scheduler after every step (alternative: 'epoch').
            "interval": "epoch",
            # Updates the learning rate after every step.
            "frequency": 1,
            # Metric to monitor for scheduler
            "monitor": "valid_loss",
            # Enforce that the value specified 'monitor' is available when the scheduler is updated, 
            # thus stopping training if not found.
            "strict": True,
            # No custom logged name
            "name": None,
        }
        return {"optimizer": optimizer, 'lr_scheduler': scheduler}

## Implement model

In [ ]:
# Implements entry to SepConv2d, see Lang et al. (2019), p. 6
class MyEntryLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        self.out_channels = out_channels

        self.proj_out = nn.Conv2d(in_channels, out_channels[len(out_channels)-1], (1,1))

        self.entry_blocks = nn.ModuleList()
        for i in range(len(out_channels)):
            self.entry_blocks.append(nn.Sequential(
                nn.Conv2d(in_channels, out_channels[i], (1, 1)),
                nn.BatchNorm2d(out_channels[i]),
                nn.ReLU(inplace = True)
            ))
            in_channels = out_channels[i]  # Update in_channels for next iteration

    def forward(self, x):
        x_entry = x
        for i in range(len(self.out_channels)):
            x_entry = self.entry_blocks[i](x_entry)
        x = self.proj_out(x)
        return (x + x_entry)

In [ ]:
# Implements SepConv2D
class MySepConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel, **kwargs):
        super().__init__()
        if in_channels == out_channels:
            self.proj_out = nn.Identity()
        else:
            self.proj_out = nn.Conv2d(in_channels, out_channels, (1,1), **kwargs)

        self.sep_conv_block = nn.Sequential(
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels, in_channels, kernel, groups=in_channels, **kwargs), # depthwise SepConv
            nn.Conv2d(in_channels, out_channels, (1,1), **kwargs), # pointwise SepConv
            nn.BatchNorm2d(out_channels)
        )
    
    def forward(self, x):
        x_sep_conv = self.sep_conv_block(x)
        x_sep_conv_2 = self.sep_conv_block(x_sep_conv) # performs second SepConv, see Lang et al. (2019), p. 6
        x = self.proj_out(x)
        return (x + x_sep_conv_2) # adds original input and sep_conv_2 output

In [ ]:
tree_model = MyCNNModel(
    MyEntryLayer(12, [32, 64, 128]), # increase number of channels to 128
    MySepConvLayer(128, 128, (3,3), padding='same'),
    MySepConvLayer(128, 128, (3,3), padding='same'),
    MySepConvLayer(128, 128, (3,3), padding='same'),
    MySepConvLayer(128, 128, (3,3), padding='same'),
    MySepConvLayer(128, 128, (3,3), padding='same'),
    MySepConvLayer(128, 128, (3,3), padding='same'),
    MySepConvLayer(128, 128, (3,3), padding='same'),
    MySepConvLayer(128, 128, (3,3), padding='same'),
    MySepConvLayer(128, 128, (3,3), padding='same'),
    MySepConvLayer(128, 128, (3,3), padding='same'),
    MySepConvLayer(128, 128, (3,3), padding='same'),
    MySepConvLayer(128, 128, (3,3), padding='same'),
    nn.AdaptiveMaxPool2d(1),
    nn.Flatten(1),
    nn.Linear(128, 1)
)

In [ ]:
# New, we need a trainer class
from pytorch_lightning.callbacks import RichProgressBar, RichModelSummary
trainer1 = pl.Trainer(devices=1, accelerator="cpu", precision='64', max_epochs=15,
                      callbacks=[RichProgressBar(refresh_rate=1),
                                 RichModelSummary(3),
                                ])

In [ ]:
trainer1.fit(tree_model, trainloader, validloader)

In [ ]:
tree_model.eval()
tree_model = tree_model.float()
batch = next(iter(trainloader))
inputs = batch[0]
inputs = inputs.float()

print("Input shape:", inputs.shape)


with torch.no_grad():
    predictions = tree_model(inputs).flatten()

true_heights = batch[1]
print("Targets:", batch[1])
print("Target shape:", batch[1].shape)
print("Predictions:", predictions)
print("Prediction shape:", predictions.shape)
# expected batch size number of predictions for height !

In [ ]:
max_val = 0
for batch in validloader:
    max_ground = max(batch[1])
    if(max_ground > max_val):
        max_val = max_ground
print("Max ground truth in validation set is: ", max_val)

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

tree_model.eval()
tree_model = tree_model.float()

num_classes = 10  # Number of size classes
class_intervals = 4  # Interval between size classes
# 10 classes of each 4 m because maximum height in trainset is 39.6 (see above)
class_thresholds = [i * class_intervals for i in range(1, num_classes+1)]

mse_total = [0.0] * num_classes
mae_total = [0.0] * num_classes
class_counts = [0] * num_classes

true_heights_total = 0.0
predictions_total = 0.0

with torch.no_grad():
    progress_bar = tqdm(validloader, desc="Evaluation")
    for batch in progress_bar:
        inputs, true_heights = batch[0].float(), batch[1].float()
        batch_size = inputs.size(0)

        predictions = tree_model(inputs)
        true_heights = true_heights.view(-1, 1)

        mse = F.mse_loss(predictions, true_heights, reduction='none').squeeze()
        mae = F.l1_loss(predictions, true_heights, reduction='none').squeeze()

        true_heights_total += true_heights.sum().item()
        predictions_total += predictions.sum().item()

        for i, threshold in enumerate(class_thresholds):
            indices = (true_heights <= threshold).squeeze(1)
            mse_total[i] += mse[indices].sum().item()
            mae_total[i] += mae[indices].sum().item()
            class_counts[i] += indices.sum().item()

        progress_bar.set_postfix({'Total MSE': mse_total[0] / class_counts[0], 'Total MAE': mae_total[0] / class_counts[0]})

mse_class_avg = [mse_total[i] / class_counts[i] if class_counts[i] != 0 else 0.0 for i in range(num_classes)]
mae_class_avg = [mae_total[i] / class_counts[i] if class_counts[i] != 0 else 0.0 for i in range(num_classes)]
average_true_height = true_heights_total / len(validloader.dataset)
average_prediction = predictions_total / len(validloader.dataset)
# Calculate overall MSE and MAE
overall_mse = sum(mse_total) / sum(class_counts) if sum(class_counts) != 0 else 0.0
overall_mae = sum(mae_total) / sum(class_counts) if sum(class_counts) != 0 else 0.0


# Print the evaluation metrics for each size class
for i, threshold in enumerate(class_thresholds):
    print(f"Size Class {i+1}:")
    print(f"MSE: {mse_class_avg[i]}")
    print(f"MAE: {mae_class_avg[i]}")

# Print the overall evaluation metrics
print("Average True Height:", average_true_height)
print("Average Prediction:", average_prediction)
print("Average MSE: ", overall_mse )
print("Average MAE:", overall_mae )

In [ ]:
res = 15
res = int((res-1)/2)
nan_values = 0
for i in range(10):
    image = np.load(f"test_images/image_00{i}.npy")
    image = np.transpose(image, (1,2,0))
    
    
    nan_values_before = (np.count_nonzero(np.isnan(image)))
            
    channel8 = image[:, :, 6]
    channel4 = image[:, :, 2]
    channels = image.shape
    width = image[0].shape[0]
    height = image[0].shape[1]

    # add the vegetation array 
    vegetation_array = np.divide((np.subtract(channel8, channel4)), np.add(np.add(channel8, channel4), 1e-6))
            
    nan_values_vegetation = (np.count_nonzero(np.isnan(vegetation_array)))
            
    if(nan_values_vegetation > 0): 
        print("picture",f"test_images/image_00{i}.npy had", nan_values_before, "before vegetation index")
        print("picture",f"test_images/image_00{i}.npy has", nan_values_vegetation, "nan_values after adding vegetation")
            
            
    vegetation_array = np.nan_to_num(vegetation_array, nan=0.0)
    image_transformed = np.concatenate((image, vegetation_array[:, :, np.newaxis]), axis=2)

            
    image = image_transformed
    nan_values_before = 0
    # add moisture index
    channel8a = image[:, :, 7]
    channel11 = image[:, :, 8]
    moisture_array = np.divide((np.subtract(channel8a, channel11)), np.add(np.add(channel8a, channel11), 1e-6))
            
    nan_values_moisture = (np.count_nonzero(np.isnan(moisture_array)))
            
    if(nan_values_moisture > 0): 
        print("picture",f"test_images/image_00{i}.npy had", nan_values_before, "before moisture index")
        print("picture",f"test_images/image_00{i}.npy has", nan_values_moisture, "nan_values after adding moisture")
            
            
    image_transformed = np.concatenate((image,moisture_array[:,:, np.newaxis]), axis = 2)
    image = image_transformed

    nan_values_pic = np.count_nonzero(np.isnan(image))
    nan_values += nan_values_pic
    
    # Add padding to every image edge in case there are ground truths which are too close to an edge
    padded_image = np.pad(image, ((res+1, res+1), (res+1, res+1), (0,0)), mode='constant')
    
    
    
    
    
    
    model = torch.load("Architecture2_0.03_10_NoAug_Model.zip")
    model.eval()
    pred = np.zeros((1024, 1024))
    for p in range(res+1, 1024+res+1):
        for q in range(res+1, 1024+res+1):
            patch = padded_image[p-res : p+res+1, q-res : q+res+1, :]
            patch = torch.from_numpy(patch).float()
            patch = torch.unsqueeze(patch, 0).permute(0,3,1,2)
            pred[p-(res+1), q-(res+1)] = model.predict(patch)
            
    
    np.save(f"test_images/prediction_00{i}.npy", pred)